In [ ]:
import huracanpy
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.geometry import Polygon, MultiPolygon, Point
import geopandas as gpd
from glob import glob

In [ ]:
ls ../TC_tracks/2D

In [ ]:
# - LOAD TRACKS
tracks={}

# UM
um_files = glob("../TC_tracks/um_*")
for f in um_files:
    name = f.split("/")[-1].split(".")[0]
    tracks[name] = huracanpy.load(f)
    tracks[name] = tracks[name].rename({"sfcwind_max":"wind", "psl_min":"slp"})

# IFS files
ifs_files = glob("../TC_tracks/ifs_*")
for f in ifs_files:
    name = f.split("/")[-1].split(".")[0]
    tracks[name] = huracanpy.load(f).rename({"psl":"slp"})

# CAS-ESM
cas_files = glob("../TC_tracks/casesm2_*")
for f in cas_files:
    name = f.split("/")[-1].split(".")[0]
    tracks[name] = huracanpy.load(f)

# ICO
icon_files = glob("../TC_tracks/2D/icon_*")
for f in icon_files:
    name = f.split("/")[-1].split(".")[0]
    tracks[name] = huracanpy.load(f).rename({"psl":"slp"})

# SCREAM
tracks["scream_ne120"] = huracanpy.load("../TC_tracks/scream_ne120.txt", source = "tempestextremes", tempest_extremes_unstructured=True)
tracks["scream_ne120"] = tracks["scream_ne120"].rename({"feature_0":"slp", "feature_1":"wind"})

In [ ]:
for n in tracks:
    tracks[n] = tracks[n].where((tracks[n].time >= np.datetime64("2020-03-01")) & (tracks[n].time <= np.datetime64("2021-02-28")), drop = True)

In [ ]:
ibtracs_full = huracanpy.load(source = "ibtracs")
ibtracs_period = ibtracs_full.where((ibtracs_full.time >= np.datetime64("2020-03-01")) & (ibtracs_full.time <= np.datetime64("2021-02-28")), drop = True)

In [ ]:
ibtracs_period["wind"] /= 1.95

In [ ]:
#tracks["ibtracs_full"] = ibtracs_full
tracks["ibtracs_2020-21"] = ibtracs_period

# Statistics

In [ ]:
NTC = {}
for n in tracks:
    t = tracks[n]
    NTC[n] = t.track_id.hrcn.nunique()
    print(n, t.track_id.hrcn.nunique())

In [ ]:
plt.bar(NTC.keys(), NTC.values())
plt.xticks(rotation=90)
plt.yticks([50, 100, 150, 200, 250, 800, 1000])
plt.grid()
plt.ylabel("#Detected TCs")

In [ ]:
TCD = {}
for n in tracks:
    t = tracks[n]
    TCD[n] = t.hrcn.get_track_duration().sum() / 24

In [ ]:
plt.bar(TCD.keys(), TCD.values())
plt.xticks(rotation=90)
#plt.yticks([50, 100, 150, 200, 250, 800, 1000])
plt.grid()
plt.ylabel("TC days")
plt.ylim(0, 3000)

In [ ]:
ACE = {}
for n in tracks:
    t = tracks[n]
    t = t.hrcn.add_ace(wind_name="wind", threshold = 0)
    ACE[n] = t.ace.sum().mean()

In [ ]:
plt.bar(ACE.keys(), ACE.values())
plt.xticks(rotation=90)
#plt.yticks([50, 100, 150, 200, 250, 800, 1000])
plt.grid()
plt.ylabel("ACE")
plt.ylim(0, 3000)

## Dstribution in basins

In [ ]:
for n in tracks:
    tracks[n] = tracks[n].hrcn.add_basin()
    tracks[n] = tracks[n].where(tracks[n].basin != '', drop = True)

In [ ]:
Nbasins = pd.DataFrame(tracks["ibtracs_2020-21"].to_dataframe().groupby("basin").track_id.nunique().sort_values(ascending = False).rename("ibtracs"))

In [ ]:
for n in tracks:
    Nbasins = Nbasins.join(tracks[n].to_dataframe().groupby("basin").track_id.nunique().sort_values(ascending = False).rename(n))

In [ ]:
for n in tracks:
    if n != 'ibtracs_2020-21':
        plt.scatter(Nbasins.index, Nbasins[n], label = n)
    if n == 'ibtracs_2020-21':
        plt.scatter(Nbasins.index, Nbasins[n], label = n, color = 'k', marker = "*", s = 100)
plt.ylim(0,75)
plt.legend(ncol=2)

In [ ]:
tracks.keys()

In [ ]:
tracks["casesm2_10km_cumulus"].where(tracks["casesm2_10km_cumulus"].basin == '', drop = True)

## Intensity

In [ ]:
for n in tracks:
    tracks[n] = tracks[n].hrcn.add_saffir_simpson_category()

In [ ]:
Ncat = pd.DataFrame(tracks["ibtracs_2020-21"].to_dataframe().groupby("saffir_simpson_category").track_id.nunique().rename("ibtracs_2020-21"))
for n in tracks:
    if n != "ibtracs_2020-21":
        Ncat = Ncat.join(tracks[n].to_dataframe().groupby("saffir_simpson_category").track_id.nunique().rename(n))

In [ ]:
Ncat.plot.bar(figsize = (15,4))
plt.ylabel("#TC")

In [ ]:
Ncat.loc[[3,4,5]].plot.bar(figsize = (12,4))
plt.ylabel("#TC")

In [ ]:
# Compute times from extremum
# Plot with seaborn
fig, axs = plt.subplots(1, sharex=True, figsize = (8,8))
for n in list(tracks.keys()):
    data = tracks[n]
    #data["time_from_slp_min"] = data.hrcn.get_time_from_apex(
    #    intensity_var_name="slp", stat="min"
    #) / 1e9 / 3600 / 24
    data["time_from_wind_max"] = data.hrcn.get_time_from_apex(
        intensity_var_name="wind",
    ) / 1e9 / 3600 / 24

    
    # SLP lifecycle
    #sns.lineplot(data=data, x="time_from_slp_min", y="slp", ax=axs[0], label = n)
    # Wind lifecycle
    sns.lineplot(data=data, x="time_from_wind_max", y="wind", ax=axs, label = n)

axs.legend(ncol = 2)
axs.set_xlim(-10,10)